# Bulk Shapefile Upload
*A notebook to read shapefiles and upload their data as [hosted feature layers](https://doc.arcgis.com/en/arcgis-online/share-maps/hosted-web-layers.htm) in ArcGIS*

In [ ]:
# common imports
import os
from arcgis.gis import *
import shutil

## User Input

* **gis**: your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/)
* **dir_path**: path to directory with the Shapefiles

In [ ]:
gis = GIS("https://www.arcgis.com", "<USERNAME>", "<PASSWORD>")

dir_path = "<DIRECTORY PATH>"

## Execution

In [ ]:
# get an array of all the shapefiles in the directory
shapefiles = [file for file in os.listdir(dir_path) if file.endswith('.shp')]
shp_paths = [os.path.join(dir_path, file) for file in shapefiles]
print("Pushing {0} shapefile(s)".format(len(shp_paths)))

In [ ]:
# create a new folder in the GIS to store the layers (if it doesn't already exist)
folder = os.path.basename(dir_path)
gis_folder = gis.content.create_folder(folder)

# handle folder already exists
gis_folder = folder if gis_folder is None else gis_folder

In [ ]:
# loop through shapefiles, zip them for upload, upload layers to GIS, move to new folder
for i, shp_path in enumerate(shp_paths):
    # create a temp dir to work in
    with tempfile.TemporaryDirectory() as temp_dir:
        # copy all shapefile files into temp dir and zip the dir.
        temp_shp_dir = os.path.join(temp_dir, os.path.splitext(os.path.basename(shp_path))[0])
        os.makedirs(temp_shp_dir)
        for ext in ['.shp', '.dbf', '.cpg', '.prj', '.sbn', '.sbx', '.shp.xml', '.shx']:
            if os.path.exists(os.path.splitext(shp_path)[0] + ext):
                shutil.copy(os.path.splitext(shp_path)[0]+ext, temp_shp_dir)
                
        shp_zip = shutil.make_archive(temp_shp_dir, 'zip', root_dir=temp_shp_dir)
        
        # publish the shapefile zip
        title = os.path.splitext(os.path.basename(shp_path))[0]
        item = gis.content.add(data=shp_zip, item_properties={
            "title": title,
            'type': 'Shapefile',
            "tags": title})
        # see also: https://developers.arcgis.com/rest/services-reference/feature-service.htm
        features_service = item.publish({
            "name": os.path.splitext(item['name'])[0],
            "hasStaticData": True,
            "layerInfo": {"capabilities": "Query"}
        })
        
        # move the newly uploaded item to the folder created earlier
        item.move(folder)
        features_service.move(folder)
    
    print("{0}/{1}".format(i + 1, len(shp_paths)))
